# BITS F312 - Neural Network and Fuzzy Logic



In [ ]:
# importing libraries required
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# connecting gdrive to access the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# finding out current working directory
!pwd

/content/drive/MyDrive/NNFL/Data_A1


In [ ]:
# changing directory to - 'drive/MyDrive/NNFL/Data_A1/'
%cd drive/MyDrive/NNFL/Data_A1/

/content/drive/MyDrive/NNFL/Data_A1


In [ ]:
# defining plotting style
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (14, 14)

In [ ]:
def metrics(Y_true, Y_pred):
    FP=0 # For counting the False Positives
    FN=0 # For counting the False Negatives
    TN=0 # For counting the True Negatives
    TP=0 # For counting the True Positives

    for i in range(len(Y_true)):
      if Y_true[i]==1:
        if Y_pred[i]==1:
          TP+=1
        else:
          FN+=1
      else:
        if Y_pred[i]==0:
          TN+=1
        else:
          FP+=1

    print('--------------------------------------------------------------------------------')
  
    print("False Positives : ", FP)
    print("False Negatives : ", FN)
    print("True Negatives  : ", TN)
    print("True Positives  : ", TP)

    print('--------------------------------------------------------------------------------')

    sens= TP/(TP+FN)
    spes = TN/(TN+FP)

    print("Sensitivity : ", sens)
    print("Specificity : ", spes)
    print("Accuracy ((TN+TP)/(TN+TP+FN+FP)) : ", ((TP+TN)/(TN+FN+TP+FP)))
    print('--------------------------------------------------------------------------------')



In [ ]:
def normalize(data):
  normalized_vec = data

  if(len(data.shape) == 1):
    mean = np.mean(data[:])
    std_dev = np.std(data[:])
    normalized_vec[:] = (normalized_vec[:] - mean)/std_dev
  else:
    for i in range(1,data.shape[1]):
      mean = np.mean(data[:, i])
      std_dev = np.std(data[:, i])
      normalized_vec[:,i] = (normalized_vec[:,i] - mean)/std_dev
  
  return normalized_vec

#Q9

In [ ]:
def LH_function_MAP(X, cov, mean):
    return ((1 / (np.power((2 * np.pi), X.shape[0] / 2) * np.sqrt(np.linalg.det(cov)))) * np.exp(-0.5 * (((X - mean)).dot((np.linalg.inv(cov)).dot((X - mean).T)))))

In [ ]:
def mean(data): 
    mean_val = []
    data = np.array(data)
    for i in range(data.shape[1]):
        mean_val.append(np.sum(data[:,i])/len(data))
    return mean_val

In [ ]:
def MAP(XVec_test,Y_Prob,mean_val,cov):
  res = []
  for i in range(len(mean_val)):
    res.append(LH_function_MAP(XVec_test,cov[i],mean_val[i])*Y_Prob[i])
  return (np.argmax(res))

In [ ]:
def fiveCVMAP(filename):
  # obtaining the data from the file
  df = pd.read_csv(filename, sep="\t", header = None, engine='python')
  df = df.sample(frac=1).reset_index(drop=True)
  # df.insert(0, 'ones', 1)


  fold_length = int((df.shape[0])/5)

  # preparing the data for 5 fold CV
  cv_val = 5

  cv_lists = [[] for i in range(0, cv_val)]
  fold_np = [[] for i in range(0, cv_val)]
  
  for i in range(cv_val):
    cv_lists[i] = df[i*fold_length:(i+1)*fold_length]
    fold_np[i] = cv_lists[i].to_numpy()

  for iteration in range(cv_val):
    test_data = None
    train_data = None
    train_list  = []
    for j in range(cv_val):
      if(i==j):
        test_data = fold_np[iteration]
      else:
        train_list.append(cv_lists[iteration])

    train_data = np.vstack(train_list)

    print('Fold: ', iteration+1)
    print('--------------------------------------------------------------------------------')
    # normalizing data
    XVec_train = normalize(train_data[:, :(train_data.shape[1]-1)])
    YVec_train = train_data[:,train_data.shape[1]-1]

    XVec_test = normalize(test_data[:, :(test_data.shape[1]-1)])
    YVec_test = test_data[:,test_data.shape[1]-1]
    MAP_CV(XVec_train, YVec_train, XVec_test, YVec_test)
    print('--------------------------------------------------------------------------------')


In [ ]:
def MAP_CV(XVec_train, YVec_train, XVec_test, YVec_test):
  Y_Prob = []
  mean_val = []
  cov = []
  for j in np.unique(YVec_train):
    Y_Prob.append(len([i for (i, val) in enumerate(YVec_train) if val == j])/len(YVec_train))
    x = np.array([XVec_train[i] for (i, val) in enumerate(YVec_train) if val == j])
    mean_val.append(mean(x));
    cov.append(np.cov(x.T))
  mean_val = np.array(mean_val)
  cov = np.array(cov)

  Y_Pred=[]
  for i in range(len(XVec_test)):
    Y_Pred.append(MAP(XVec_test[i],Y_Prob,mean_val,cov)+1)

  Y_Pred=np.array(Y_Pred)
  df_confusion = pd.crosstab(YVec_test, Y_Pred)

  df_confusion=np.array(df_confusion)
  row_sum=0.0;
  overall_sum=0.0;
  dia_el=0.0

  print("\n Confusion Matrix\n", df_confusion)

  for i in range(df_confusion.shape[0]):
    row_sum=np.sum(df_confusion[i,:])
    overall_sum +=row_sum
    acc=(df_confusion[i,i]/row_sum)
    dia_el+=df_confusion[i,i]
    print("Accuracy for class "+str(i+1)+" is {}".format(acc))
  

  ov_acc=((dia_el/overall_sum))
  print("Overall Accuracy for the model is {}".format(ov_acc))

  

In [ ]:
fiveCVMAP('data_q6_q7.txt')

Fold:  1
--------------------------------------------------------------------------------

 Confusion Matrix
 [[14  0  0]
 [ 0 10  0]
 [ 0  0 18]]
Accuracy for class 1 is 1.0
Accuracy for class 2 is 1.0
Accuracy for class 3 is 1.0
Overall Accuracy for the model is 1.0
--------------------------------------------------------------------------------
Fold:  2
--------------------------------------------------------------------------------

 Confusion Matrix
 [[16  0  0]
 [ 0 13  0]
 [ 0  0 13]]
Accuracy for class 1 is 1.0
Accuracy for class 2 is 1.0
Accuracy for class 3 is 1.0
Overall Accuracy for the model is 1.0
--------------------------------------------------------------------------------
Fold:  3
--------------------------------------------------------------------------------

 Confusion Matrix
 [[14  0  0]
 [ 0 19  0]
 [ 0  0  9]]
Accuracy for class 1 is 1.0
Accuracy for class 2 is 1.0
Accuracy for class 3 is 1.0
Overall Accuracy for the model is 1.0
-------------------------------